In [1]:
import itertools
import json
from pathlib import Path

import pandas as pd

In [2]:
def sorted_tuple(x):
    return tuple(sorted(x))

In [3]:
from bellem.dvc.experiment import load_experiments

filepaths = list(Path("../../tmp/drop-mini-manual").glob("*.json"))
experiments = [exp for fp in filepaths for exp in load_experiments(fp)]
print(f"{len(experiments)} experiments")
next(iter(experiments), None)

13 experiments


{'commit': 'workspace',
 'id': 'workspace',
 'name': None,
 'params': {'dataset': {'path': 'bdsaglam/drop-musique-mini',
   'name': 'default',
   'split': 'train'},
  'qa': {'model': 'llama-3-8b',
   'temperature': 0.0,
   'top_p': 0.9999,
   'system_prompt': 'direct/format-few.txt',
   'user_prompt_template': 'cq.txt',
   'few_shot_examples': 'manual/direct.json',
   'n_shot': 0,
   'n_sc': 1},
  'run': 1},
 'metrics': {'exact_match': 0.37,
  'f1': 0.5993370884585718,
  'fuzzy_match': 0.41,
  '0hops': {'exact_match': 0.37,
   'f1': 0.5993370884585718,
   'fuzzy_match': 0.41},
  'gen_token_count': {'all': {'count': 100.0,
    'mean': 6.41,
    'std': 4.431510219810265,
    'min': 3.0,
    '25%': 4.0,
    '50%': 5.0,
    '75%': 7.0,
    'max': 28.0},
   'success': {'count': 65.0,
    'mean': 5.3076923076923075,
    'std': 2.8718628368789054,
    'min': 3.0,
    '25%': 4.0,
    '50%': 5.0,
    '75%': 5.0,
    'max': 22.0},
   'fail': {'count': 35.0,
    'mean': 8.457142857142857,
    'st

In [4]:
df = pd.json_normalize(experiments)
df.drop(columns=[col for col in df.columns if "fuzzy" in col], inplace=True)
print(f"{len(df)} experiments before preprocessing")
df.head()

13 experiments before preprocessing


,commit,id,name,params.dataset.path,params.dataset.name,params.dataset.split,params.qa.model,params.qa.temperature,params.qa.top_p,params.qa.system_prompt,...,metrics.gen_token_count.success.75%,metrics.gen_token_count.success.max,metrics.gen_token_count.fail.count,metrics.gen_token_count.fail.mean,metrics.gen_token_count.fail.std,metrics.gen_token_count.fail.min,metrics.gen_token_count.fail.25%,metrics.gen_token_count.fail.50%,metrics.gen_token_count.fail.75%,metrics.gen_token_count.fail.max
0,workspace,workspace,None,bdsaglam/drop-musique-mini,default,train,llama-3-8b,0.0,0.9999,direct/format-few.txt,...,5.00,22.0,35.0,8.457143,5.917784,3.0,5.0,6.0,9.00,28.0
1,ef6fb76896f7144e8b2450953e6096000221940d,e44b68f858451df3b29f77987bd3e502732c7a7d,idled-site,bdsaglam/drop-musique-mini,default,train,llama-3.3-70b,0.0,0.9990,direct/format-few-drop.txt,...,4.00,18.0,33.0,6.090909,5.832900,1.0,3.0,4.0,7.00,32.0
2,ef6fb76896f7144e8b2450953e6096000221940d,5a563d86a7065f96759f6784dca4298626801a46,fated-fuss,bdsaglam/drop-musique-mini,default,train,llama-3.3-70b,0.0,0.9990,ccot/format-thought-drop.txt,...,57.75,203.0,22.0,65.454545,54.838054,22.0,28.0,42.0,83.25,210.0
3,ef6fb76896f7144e8b2450953e6096000221940d,b1968b921ffb6b8b8b50657b9c8af4f0d7177a0a,chewy-choc,bdsaglam/drop-musique-mini,default,train,llama-3.3-70b,0.0,0.9990,direct/format-few-drop.txt,...,4.50,18.0,33.0,6.121212,5.818922,1.0,3.0,4.0,7.00,32.0
4,ef6fb76896f7144e8b2450953e6096000221940d,82ce134808faec0474ec47c86f03d9f62f807e11,cronk-doit,bdsaglam/drop-musique-mini,default,train,llama-3.3-70b,0.5,0.9500,direct/format-few-drop.txt,...,5.00,18.0,31.0,6.612903,6.897717,1.0,3.0,4.0,6.50,32.0


In [5]:
if 'params.qa.top_p' not in df.columns:
    df['params.qa.top_p'] = None

In [6]:
param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]

In [7]:
df.dropna(subset=metric_cols[:2], inplace=True, how="any")
df.drop_duplicates(subset=param_cols, keep='last', inplace=True)

print(f"{len(df)} experiments after preprocessing")
df.head()

13 experiments after preprocessing


,commit,id,name,params.dataset.path,params.dataset.name,params.dataset.split,params.qa.model,params.qa.temperature,params.qa.top_p,params.qa.system_prompt,...,metrics.gen_token_count.success.75%,metrics.gen_token_count.success.max,metrics.gen_token_count.fail.count,metrics.gen_token_count.fail.mean,metrics.gen_token_count.fail.std,metrics.gen_token_count.fail.min,metrics.gen_token_count.fail.25%,metrics.gen_token_count.fail.50%,metrics.gen_token_count.fail.75%,metrics.gen_token_count.fail.max
0,workspace,workspace,None,bdsaglam/drop-musique-mini,default,train,llama-3-8b,0.0,0.9999,direct/format-few.txt,...,5.00,22.0,35.0,8.457143,5.917784,3.0,5.0,6.0,9.00,28.0
1,ef6fb76896f7144e8b2450953e6096000221940d,e44b68f858451df3b29f77987bd3e502732c7a7d,idled-site,bdsaglam/drop-musique-mini,default,train,llama-3.3-70b,0.0,0.9990,direct/format-few-drop.txt,...,4.00,18.0,33.0,6.090909,5.832900,1.0,3.0,4.0,7.00,32.0
2,ef6fb76896f7144e8b2450953e6096000221940d,5a563d86a7065f96759f6784dca4298626801a46,fated-fuss,bdsaglam/drop-musique-mini,default,train,llama-3.3-70b,0.0,0.9990,ccot/format-thought-drop.txt,...,57.75,203.0,22.0,65.454545,54.838054,22.0,28.0,42.0,83.25,210.0
3,ef6fb76896f7144e8b2450953e6096000221940d,b1968b921ffb6b8b8b50657b9c8af4f0d7177a0a,chewy-choc,bdsaglam/drop-musique-mini,default,train,llama-3.3-70b,0.0,0.9990,direct/format-few-drop.txt,...,4.50,18.0,33.0,6.121212,5.818922,1.0,3.0,4.0,7.00,32.0
4,ef6fb76896f7144e8b2450953e6096000221940d,82ce134808faec0474ec47c86f03d9f62f807e11,cronk-doit,bdsaglam/drop-musique-mini,default,train,llama-3.3-70b,0.5,0.9500,direct/format-few-drop.txt,...,5.00,18.0,31.0,6.612903,6.897717,1.0,3.0,4.0,6.50,32.0


In [8]:
def parse_qa_technique(row):
    model = row['params.qa.model']
    if 'r1' in model:
        base_technique = 'REASONING'
    else:
        base_technique = row["params.qa.system_prompt"].split("/")[0].replace(".txt", "").upper()
    sc = int(row["params.qa.n_sc"])
    sc_suffix = f"-SC({sc})" if sc > 1 else ""
    return f"{base_technique}{sc_suffix}"

df["params.qa.technique"] = df.apply(parse_qa_technique, axis=1)

In [9]:
def remove_ext(filename):
    return filename.split(".", 1)[0]

def parse_instruction(row):
    sp = remove_ext(row["params.qa.system_prompt"]).lower()
    up = remove_ext(row["params.qa.user_prompt_template"]).lower().replace(".txt", "")
    fs = remove_ext(row["params.qa.few_shot_examples"]).lower()
    return f"{sp}:{up}:{fs}"

if len(df):
    df["params.qa.instruction"] = df.apply(parse_instruction, axis=1)

In [10]:
# make temperature 1 decimal
if len(df):
    df["params.qa.temperature"] = df["params.qa.temperature"].round(1)

In [11]:
for col in param_cols:
    values = list(df[col].unique())
    print(f"- {col}: {values}")
    print()

- params.dataset.path: ['bdsaglam/drop-musique-mini']

- params.dataset.name: ['default']

- params.dataset.split: ['train']

- params.qa.model: ['llama-3-8b', 'llama-3.3-70b']

- params.qa.temperature: [0.0, 0.5]

- params.qa.top_p: [0.9999, 0.999, 0.95]

- params.qa.system_prompt: ['direct/format-few.txt', 'direct/format-few-drop.txt', 'ccot/format-thought-drop.txt']

- params.qa.user_prompt_template: ['cq.txt']

- params.qa.few_shot_examples: ['manual/direct.json', 'drop/direct.json', 'drop/ccot.json']

- params.qa.n_shot: [0]

- params.qa.n_sc: [1]

- params.run: [1, 3, 2]



In [12]:
df.to_json('exps.jsonl', orient='records', lines=True)

## Setup remaining experiments

In [13]:
def produce_experiment_configs(common_params, varying_params):
    # Generate all possible combinations of parameters
    varying_params = {**common_params, **varying_params}
    keys = varying_params.keys()
    values = varying_params.values()
    for instance in itertools.product(*values):
        yield dict(zip(keys, instance))

In [14]:
def produce_all_experiment_configs(common_params: dict, varying_params_list: list[dict]):
    for params in varying_params_list:
        for exp_config in produce_experiment_configs(common_params, params):
            yield exp_config

In [15]:
common_params = {
    "params.dataset.path": ["bdsaglam/drop-musique-mini"],
    "params.dataset.split": ["train"],
    "params.qa.n_sc": [
        1,
    ],
    "params.qa.n_shot": [
        0,
        # 1,
    ],
    "params.run": [
        1,
        2,
        3,
    ],
}

In [16]:
varying_params_list = [
    {
        "params.qa.model": [
            # "llama-3-8b",
            "llama-3.3-70b",
        ],
        "params.qa.temperature": [
            0.0
        ],
        "params.qa.top_p": [0.999],
        "params.qa.system_prompt": ["direct/format-few-drop.txt"],
        "params.qa.few_shot_examples": ["drop/direct.json"],
    },
    {
        "params.qa.model": [
            # "llama-3-8b",
            "llama-3.3-70b",
        ],
        "params.qa.temperature": [
            0.5,
        ],
        "params.qa.top_p": [
            0.95,
        ],
        "params.qa.system_prompt": ["direct/format-few-drop.txt"],
        "params.qa.few_shot_examples": ["drop/direct.json"],
    },
    {
        "params.qa.model": [
            # "llama-3-8b",
            "llama-3.3-70b",
        ],
        "params.qa.temperature": [
            0.0,
        ],
        "params.qa.top_p": [0.999],
        "params.qa.system_prompt": ["ccot/format-thought-drop.txt"],
        "params.qa.few_shot_examples": ["drop/ccot.json"],
    },
    {
        "params.qa.model": [
            # "llama-3-8b",
            "llama-3.3-70b",
        ],
        "params.qa.temperature": [
            0.5,
        ],
        "params.qa.top_p": [
            0.95,
        ],
        "params.qa.system_prompt": ["ccot/format-thought-drop.txt"],
        "params.qa.few_shot_examples": ["drop/ccot.json"],
    },
    {
        "params.qa.model": [
            # "deepseek-r1-llama-8b",
            "deepseek-r1-llama-70b",
        ],
         "params.qa.temperature": [
            0.5,
        ],
        "params.qa.top_p": [
            0.95,
        ],
        "params.qa.system_prompt": ["empty.txt"],
        "params.qa.user_prompt_template": ["icq-format-drop.txt"],
        "params.qa.few_shot_examples": ["drop/reasoning.json"],
    },
]

In [17]:
exp_configs = list(produce_all_experiment_configs(common_params, varying_params_list))
target_params = list(exp_configs[0].keys())
print(f"{len(exp_configs)} experiment configurations")
print(target_params)

15 experiment configurations
['params.dataset.path', 'params.dataset.split', 'params.qa.n_sc', 'params.qa.n_shot', 'params.run', 'params.qa.model', 'params.qa.temperature', 'params.qa.top_p', 'params.qa.system_prompt', 'params.qa.few_shot_examples']


In [18]:
if len(df):
    existing_configs = df[target_params].to_dict(orient="records")
    existing_configs[0]
else:
    existing_configs = []

print("Existing exps:", len(existing_configs))

Existing exps: 13


In [19]:
# find the missing configurations
missing_configs = [
    dict(kv)
    for kv in list(
        {tuple(sorted(config.items())) for config in exp_configs}
        - {tuple(sorted(config.items())) for config in existing_configs}
    )
]
print(f"{len(missing_configs)} missing configurations")

3 missing configurations


In [20]:
def make_command(exp_config):
    lines = ["dvc exp run --queue"]
    for target_param in target_params:
        arg_name = target_param.split(".", 1)[-1]
        arg_value = exp_config[target_param]
        if isinstance(arg_value, str) and '[' in arg_value:
            arg_value = f'"{arg_value}"'
        lines.append(f"-S {arg_name}='{arg_value}'")

    command = " \\\n    ".join(lines)
    return command

In [21]:
with open("run.sh", "w") as f:
    f.write("#!/bin/sh\n\n")
    for exp_config in missing_configs:
        f.write(make_command(exp_config))
        f.write("\n\n")

## Inspect

In [22]:
df

,commit,id,name,params.dataset.path,params.dataset.name,params.dataset.split,params.qa.model,params.qa.temperature,params.qa.top_p,params.qa.system_prompt,...,metrics.gen_token_count.fail.count,metrics.gen_token_count.fail.mean,metrics.gen_token_count.fail.std,metrics.gen_token_count.fail.min,metrics.gen_token_count.fail.25%,metrics.gen_token_count.fail.50%,metrics.gen_token_count.fail.75%,metrics.gen_token_count.fail.max,params.qa.technique,params.qa.instruction
0,workspace,workspace,None,bdsaglam/drop-musique-mini,default,train,llama-3-8b,0.0,0.9999,direct/format-few.txt,...,35.0,8.457143,5.917784,3.0,5.00,6.0,9.00,28.0,DIRECT,direct/format-few:cq:manual/direct
1,ef6fb76896f7144e8b2450953e6096000221940d,e44b68f858451df3b29f77987bd3e502732c7a7d,idled-site,bdsaglam/drop-musique-mini,default,train,llama-3.3-70b,0.0,0.9990,direct/format-few-drop.txt,...,33.0,6.090909,5.832900,1.0,3.00,4.0,7.00,32.0,DIRECT,direct/format-few-drop:cq:drop/direct
2,ef6fb76896f7144e8b2450953e6096000221940d,5a563d86a7065f96759f6784dca4298626801a46,fated-fuss,bdsaglam/drop-musique-mini,default,train,llama-3.3-70b,0.0,0.9990,ccot/format-thought-drop.txt,...,22.0,65.454545,54.838054,22.0,28.00,42.0,83.25,210.0,CCOT,ccot/format-thought-drop:cq:drop/ccot
3,ef6fb76896f7144e8b2450953e6096000221940d,b1968b921ffb6b8b8b50657b9c8af4f0d7177a0a,chewy-choc,bdsaglam/drop-musique-mini,default,train,llama-3.3-70b,0.0,0.9990,direct/format-few-drop.txt,...,33.0,6.121212,5.818922,1.0,3.00,4.0,7.00,32.0,DIRECT,direct/format-few-drop:cq:drop/direct
4,ef6fb76896f7144e8b2450953e6096000221940d,82ce134808faec0474ec47c86f03d9f62f807e11,cronk-doit,bdsaglam/drop-musique-mini,default,train,llama-3.3-70b,0.5,0.9500,direct/format-few-drop.txt,...,31.0,6.612903,6.897717,1.0,3.00,4.0,6.50,32.0,DIRECT,direct/format-few-drop:cq:drop/direct
5,ef6fb76896f7144e8b2450953e6096000221940d,dac477c08368caacce97477b771b52d24accb5f9,inert-pein,bdsaglam/drop-musique-mini,default,train,llama-3.3-70b,0.5,0.9500,direct/format-few-drop.txt,...,33.0,5.969697,5.681576,1.0,3.00,4.0,7.00,32.0,DIRECT,direct/format-few-drop:cq:drop/direct
6,ef6fb76896f7144e8b2450953e6096000221940d,5876ba42d71f494a771f2993a3814983e65f5fb4,balky-mine,bdsaglam/drop-musique-mini,default,train,llama-3.3-70b,0.5,0.9500,ccot/format-thought-drop.txt,...,25.0,57.280000,38.045499,23.0,33.00,49.0,57.00,164.0,CCOT,ccot/format-thought-drop:cq:drop/ccot
7,ef6fb76896f7144e8b2450953e6096000221940d,5fd08560c78f78397a0f7773e32ac42a4015223b,tuned-toff,bdsaglam/drop-musique-mini,default,train,llama-3.3-70b,0.5,0.9500,ccot/format-thought-drop.txt,...,23.0,54.260870,27.057544,22.0,32.50,49.0,67.50,113.0,CCOT,ccot/format-thought-drop:cq:drop/ccot
8,ef6fb76896f7144e8b2450953e6096000221940d,2b1f812fae006dc778b2b83b9de645da4190f259,stony-bunt,bdsaglam/drop-musique-mini,default,train,llama-3.3-70b,0.0,0.9990,direct/format-few-drop.txt,...,32.0,6.062500,5.902091,1.0,3.00,4.0,7.00,32.0,DIRECT,direct/format-few-drop:cq:drop/direct
9,ef6fb76896f7144e8b2450953e6096000221940d,3de055f9e946c96d16a8ef44f38987ad261d8ec3,bifid-tint,bdsaglam/drop-musique-mini,default,train,llama-3.3-70b,0.0,0.9990,ccot/format-thought-drop.txt,...,23.0,65.347826,54.227307,22.0,27.50,42.0,79.50,210.0,CCOT,ccot/format-thought-drop:cq:drop/ccot
